In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

In [2]:
from keras.datasets import cifar10

In [3]:
#importing dataset
(X_train,Y_train),(X_test,Y_test) = cifar10.load_data()

In [4]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train/=255 #Standardise the data
X_test/=255

In [5]:
Y_train = tf.keras.utils.to_categorical(Y_train)
Y_test = tf.keras.utils.to_categorical(Y_test)

In [6]:
Y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [7]:
#50000 train examples each of size 32 x 32 x 3
#10000 test examples each of size 32 x 32 x 3

Image size : 32 x 32 x 3

Training set size : 50000

Test set size     : 10000

NETWORK ARCHITECTURE:

The network has 3 convolutional blocks.

Block 1 : 

3 Convolutional Layers each having 32 filters of size 3 x 3. 
The padding is 'same'.Each filter is initialised using 'he_uniform' initializer.
Relu activation has been used.Batch Normalization is applied at the end of each layer.

Max Pooling : Size 2 x 2 with stride 2.

Droupout Regularisation : Drop Probability = 0.5

Block 2 : 

2 Convolutional Layers each having 64 filters of size 5 x 5. 
The padding is 'same'.Each filter is initialised using 'he_uniform' initializer.
Relu activation has been used.Batch Normalization is applied at the end of each layer.

Max Pooling : Size 2 x 2 with stride 2.

Droupout Regularisation : Drop Probability = 0.5

Block 3 :

2 Convolutional Layers each having 256 filters of size 5 x 5. 
The padding is 'same'.Each filter is initialised using 'he_uniform' initializer.
Relu activation has been used.Batch Normalization is applied at the end of the block.

Max Pooling : size 2 x 2 with stride 2

Droupout Regularisation : Drop Probability = 0.5

After this , the layers are flattened and fed into Fully Connected Layers

There are 3 Fully Connected Layers.Each of them is followed by Batch Normalization(except the last layer which is the output layer).

Layer 1 : 128 neurons, relu activation, he_uniform initializer

Layer 2 : 32 neurons, relu activation, he_uniform initializer

Layer 3 : 10 neurons(representing 10 classes of the dataset).It has a softmax output.

In [8]:
model = keras.Sequential()

#Block 1 : (CONV->Relu->Batch_Norm) X 3

model.add(tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),padding='same',kernel_initializer='he_uniform',input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),padding='same',kernel_initializer='he_uniform'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),padding='same',kernel_initializer='he_uniform'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())

#Pooling and Droupout

model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2))
model.add(tf.keras.layers.Dropout(0.50))

#Block 2 : (CONV->Relu->Batch_Norm) X 2 

model.add(tf.keras.layers.Conv2D(filters=64,kernel_size=(5,5),padding='same',kernel_initializer='he_uniform'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(filters=64,kernel_size=(5,5),padding='same',kernel_initializer='he_uniform'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())

#Pooling and Droupout

model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2))
model.add(tf.keras.layers.Dropout(0.50))

#Block 3 : (CONV->Relu->Batch_Norm) X 2

model.add(tf.keras.layers.Conv2D(filters=256,kernel_size=(5,5),padding='same',kernel_initializer='he_uniform'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(filters=256,kernel_size=(5,5),padding='same',kernel_initializer='he_uniform'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())

#Pooling and Droupout

model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2))
model.add(tf.keras.layers.Dropout(0.50))

#Fully Connected Layers : (Dense_layer(Relu)->Batch_Norm)

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128,activation='relu',kernel_initializer='he_uniform'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(32,activation='relu',kernel_initializer='he_uniform'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(10,activation="softmax"))


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9

IMAGE AUGMENTATION:

The Trainset is fed into ImageDataGenerator for augmentation.

Rotation range : 15

Width and Height shift range : 0.2

Zoom range : 0.2

Horizontal flipping is allowed

In [10]:
image = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=15, width_shift_range=0.2,height_shift_range=0.2,zoom_range=0.2,horizontal_flip=True)
image.fit(X_train)

In [11]:
X_train.shape

(50000, 32, 32, 3)

Loss function : Categorical crossentropy

Optimizer     : Adam , learning rate = 0.013, all other parameters remain the same.

Batch size    : 64

epochs        : 200

In [12]:
opt = keras.optimizers.Adam(learning_rate=0.013)
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
#model.fit(X_train,Y_train,validation_split=0.2,batch_size=1024,epochs=200)
model.fit_generator(image.flow(X_train,Y_train,batch_size=64),steps_per_epoch=X_train.shape[0]/64,epochs=200,validation_data=(X_test,Y_test))

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/200
782/781 [==============================] - 29s 37ms/step - loss: 1.7030 - accuracy: 0.3809 - val_loss: 1.7430 - val_accuracy: 0.4268
Epoch 2/200
782/781 [==============================] - 28s 36ms/step - loss: 1.3488 - accuracy: 0.5138 - val_loss: 1.2036 - val_accuracy: 0.5793
Epoch 3/200
782/781 [==============================] - 29s 37ms/step - loss: 1.1581 - accuracy: 0.5890 - val_loss: 1.0722 - val_accuracy: 0.6347
Epoch 4/200
782/781 [==============================] - 28s 36ms/step - loss: 1.0309 - accuracy: 0.6368 - val_loss: 0.8860 - val_accuracy: 0.6883
Epoch 5/200
782/781 [==============================] - 28s 36ms/step - loss: 0.9536 - accuracy: 0.6670 - val_loss: 0.8589 - val_accuracy: 0.7069
Epoch 6/200
782/781 [==============================] - 28s 36ms/step - loss: 0.8867 - accuracy: 0.6894 - val_loss: 0.7908 - val_accuracy: 0.7306
Epoch 7/200
782/781 [==============================] -

In [13]:
  model.evaluate(X_test, Y_test, batch_size=128)

79/79 [==============================] - 1s 10ms/step - loss: 0.3192 - accuracy: 0.8978


[0.31920865178108215, 0.8978000283241272]

We obtain 89.92 % Train accuracy and 89.78% Test accuracy